In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
from input_configuration import base_year

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [5]:
output_dir = r'..\..\..\outputs\compare\RTP\person'

# Trips per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [6]:
pd.options.display.float_format = '{:0,.1f}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\person_trips.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_trip['is_rgc'] = 0
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

### County and Region

In [7]:
def trips_per_day(geog): 
    df1 = df_person.groupby(geog).sum()[['psexpfac']]
    df2 = df_trip.groupby(geog).sum()[['trexpfac']]
    df = df1.merge(df2, left_index=True, right_index=True)
    df.loc['Region', 'psexpfac'] = df_person.sum()['psexpfac']
    df.loc['Region', 'trexpfac'] = df_trip.sum()['trexpfac']
    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']

    df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
    for purp in ['Work','Non-Work']:
        _df = df_trip[df_trip['trip_type'] == purp].groupby(geog).sum()[['trexpfac']]
        _df.rename(columns={'trexpfac': purp + ' Trips'}, inplace=True)

        # Add regional total to bottom of table
        _df.loc['Region', purp + ' Trips'] = df_trip[df_trip['trip_type'] == purp].sum()['trexpfac']
        df = df.merge(_df, left_index=True, right_index=True)

    df['Work Trips per Day'] = df['Work Trips']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work Trips']/df['psexpfac']
    df[['Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']]
    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [8]:
df = trips_per_day('hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_county.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.7,3.5,4.2
Kitsap,0.5,3.4,3.9
Pierce,0.5,3.3,3.8
Snohomish,0.6,3.4,3.9
Region,0.6,3.4,4.0


### Regional Growth Centers

In [93]:
df = trips_per_day('is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
Not in RGC,0.6,3.4,4.0
In RGC,0.9,3.7,4.6
Region,0.6,3.4,4.0


In [9]:
df = trips_per_day('hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_rgc.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.5,3.5,4.0
Ballard-Interbay,1.0,3.5,4.5
Bellevue,0.9,3.6,4.6
Bothell Canyon Park,0.8,3.2,4.0
Bremerton,0.6,3.5,4.1
Burien,0.7,3.4,4.1
Cascade,0.5,3.3,3.8
Duwamish,0.8,3.5,4.2
Everett,0.6,3.5,4.2


### Regional Geographies

In [10]:
df = trips_per_day('hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_reg_geog.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
CitiesTowns,0.5,3.4,3.9
Core,0.6,3.4,4.0
HCT,0.6,3.4,4.0
Metro,0.7,3.6,4.3
Rural,0.5,3.2,3.7
UU,0.5,3.3,3.8
Region,0.6,3.4,4.0


### Equity Geographies

In [11]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 1].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']
df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_disability_reg,0.5,3.4,3.9
hh_elderly_reg,1.7,3.4,5.1
hh_english_reg,0.6,3.5,4.1
hh_poverty_50,0.5,3.4,3.9
hh_poverty_reg,0.6,3.4,4.0
hh_racial_50,0.6,3.5,4.1
hh_racial_reg,0.6,3.5,4.1
hh_youth_reg,0.5,3.4,3.9


In [16]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 0].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']

df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df['Equity Group'] = ['NOT in '+i for i in df['Equity Group']]
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
NOT in hh_disability_reg,0.5,2.9,3.3
NOT in hh_elderly_reg,1.5,2.9,4.4
NOT in hh_english_reg,0.4,2.2,2.5
NOT in hh_poverty_50,0.0,0.1,0.1
NOT in hh_poverty_reg,0.4,2.4,2.8
NOT in hh_racial_50,0.2,0.9,1.1
NOT in hh_racial_reg,0.5,2.8,3.3
NOT in hh_youth_reg,0.6,4.0,4.7


# Miles Driven per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [110]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

### County

In [99]:
df = vmt_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_county.csv')); incr+=1
df

,Average Miles per Person
hh_county,
King,15.9
Kitsap,15.2
Pierce,17.7
Snohomish,19.8
Region,17.0


### Regional Growth Centers

In [113]:
df = vmt_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Average Miles per Person
Not in RGC,17.6
In RGC,7.1
Region,17.0


In [66]:
df = vmt_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_rgc.csv')); incr+=1
df

,Average Miles per Person
hh_rgc,
Auburn,12.0
Ballard-Interbay,10.5
Bellevue,7.2
Bothell Canyon Park,20.2
Bremerton,5.3
Burien,14.0
Cascade,17.3
Duwamish,10.8
Everett,9.2


### Regional Geography

In [67]:
df = vmt_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_reg_geog.csv')); incr+=1
df

,Average Miles per Person
hh_rg_proposed,
CitiesTowns,21.7
Core,16.3
HCT,17.3
Metro,11.6
Rural,26.0
UU,19.3
Region,17.0


### Equity Geographies
Average Miles per Person (by residence inside or outside equity geography) 

In [72]:
result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
df_in = result_df.copy()

result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
# result_df.index = ['NOT in '+i for i in result_df.index]


df_out = result_df.copy()

df = df_in.merge(df_out, left_index=True, right_index=True)
df.columns = ['Inside Equity Geography',  'Outside Equity Geography']
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_equity.csv')); incr+=1
df

,Inside Equity Geography,Outside Equity Geography
source,,
hh_youth_reg,18.7,15.0
hh_elderly_reg,17.6,16.5
hh_english_reg,15.6,17.9
hh_racial_reg,15.1,18.6
hh_racial_50,14.6,17.6
hh_poverty_reg,15.2,18.3
hh_poverty_50,11.6,17.1
hh_disability_reg,16.9,17.1


# Miles Walking and Biking per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [25]:
# df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
# df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')



# def walk_bike_per_person_time(df_vmt, df_person, geog):
    
#     results_df = pd.DataFrame()
#     for mode in ['Walk','Bike']:
        
#         _df_vmt = df_vmt[df_vmt['mode'] == mode]
#         _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
#         _df_person = df_person.groupby(geog).sum()[['psexpfac']]

#         df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
#         df.loc['Region',:] = df.sum(axis=0)
#         df['Average Miles per Person'] = df['travtime_wt']/df['psexpfac']
#         results_df[mode] = df['Average Miles per Person']
    
#     _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
#     _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
#     _df_person = df_person.groupby(geog).sum()[['psexpfac']]

#     df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
#     df.loc['Region',:] = df.sum(axis=0)
#     df['Average Miles per Person'] = df['travtime_wt']/df['psexpfac']
#     results_df['Walk and Bike'] = df['Average Miles per Person']
    
#     return results_df

# df = walk_bike_per_person_time(df_vht, df_person, 'hh_county')
# # df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
# df

In [26]:
# df = pd.read_csv(r'..\..\..\outputs\agg\dash\trip_distance.csv')
# _df = df[df['mode'] == 'Bike']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [27]:
# _df = df[df['mode'] == 'Walk']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [114]:
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1


def walk_bike_per_person(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
        _df_person = df_person.groupby(geog).sum()[['psexpfac']]

        df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

### County

In [115]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,0.5,0.3,0.8
Kitsap,0.4,0.2,0.7
Pierce,0.4,0.3,0.7
Snohomish,0.4,0.3,0.7
Region,0.5,0.3,0.8


### Regional Growth Center 

In [117]:
df = walk_bike_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'Inside RGC', 'Region']
df

,Walk,Bike,Walk and Bike
Not in RGC,0.4,0.3,0.7
Inside RGC,1.1,0.3,1.4
Region,0.5,0.3,0.8


In [76]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_rgc.csv'))
df

,Walk,Bike,Walk and Bike
hh_rgc,,,
Auburn,0.6,0.4,1.0
Ballard-Interbay,0.7,0.5,1.3
Bellevue,1.0,0.3,1.2
Bothell Canyon Park,0.4,0.5,1.0
Bremerton,0.8,0.3,1.1
Burien,0.5,0.4,0.9
Cascade,0.4,0.2,0.7
Duwamish,0.8,0.2,1.0
Everett,0.8,0.3,1.1


### Regional Geography

In [81]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_reg_geog.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,0.4,0.2,0.6
Core,0.4,0.3,0.7
HCT,0.4,0.3,0.7
Metro,0.7,0.3,1.0
Rural,0.3,0.3,0.6
UU,0.4,0.2,0.6
Region,0.5,0.3,0.8


### Equity Geographies

In [83]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
source,,,
hh_youth_reg,0.3,0.4,0.7
hh_elderly_reg,0.3,0.4,0.7
hh_english_reg,0.3,0.5,0.8
hh_racial_reg,0.3,0.5,0.8
hh_racial_50,0.3,0.5,0.8
hh_poverty_reg,0.3,0.5,0.8
hh_poverty_50,0.3,0.6,0.9
hh_disability_reg,0.3,0.5,0.7


In [84]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.index = ['NOT in '+i for i in result_df.index]
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
NOT in hh_youth_reg,0.3,0.6,0.9
NOT in hh_elderly_reg,0.3,0.5,0.8
NOT in hh_english_reg,0.3,0.5,0.7
NOT in hh_racial_reg,0.3,0.4,0.7
NOT in hh_racial_50,0.3,0.5,0.7
NOT in hh_poverty_reg,0.3,0.5,0.7
NOT in hh_poverty_50,0.3,0.5,0.8
NOT in hh_disability_reg,0.3,0.5,0.8


# Jobs Accessible within 45 Minutes of Transit

In [125]:
pd.options.display.float_format = '{:0,.0f}'.format
df = pd.read_csv(r'..\..\..\outputs\access\transit_jobs_access.csv'); incr+=1
df_lu = pd.read_csv(r'..\..\..\inputs\scenario\landuse\parcels_urbansim.txt',
                   delim_whitespace=True, usecols=['PARCELID','EMPTOT_P'])
tot_jobs = df_lu['EMPTOT_P'].sum()

### Region

In [119]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_region.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Jobs within 45-minute Transit Commute,% Total Jobs
"135,753",6.6%


### County

In [120]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_county.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
King,"228,951",11.2%
Kitsap,"3,853",0.2%
Pierce,"21,227",1.0%
Snohomish,"27,504",1.3%


### Regional Growth Center

In [24]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_rgc.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Auburn,"58,342",2.8%
Ballard-Interbay,"515,038",25.1%
Bellevue,"474,024",23.1%
Bothell Canyon Park,"40,840",2.0%
Bremerton,"28,963",1.4%
Burien,"180,823",8.8%
Cascade,"6,107",0.3%
Duwamish,"496,215",24.2%
Everett,"62,892",3.1%


### Regional Geography

In [25]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_reg_geog.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
CitiesTowns,"14,916",0.7%
Core,"57,381",2.8%
HCT,"38,223",1.9%
Metro,"336,026",16.4%
Rural,395,0.0%
UU,"8,093",0.4%


### Equity Geographies

In [26]:
geo_list = []
equity_geogs = ['youth','elderly','english','racial','poverty','disability']
for equity_geog in equity_geogs:
    for geog_type in ['_geog_vs_reg_total','_geog_vs_50_percent']:
        if geog_type == '_geog_vs_reg_total' or equity_geog in ['poverty','racial']:
            geo_list.append(equity_geog+geog_type)

In [27]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
youth_geog_vs_reg_total,"49,746",2.4%
elderly_geog_vs_reg_total,"108,040",5.3%
english_geog_vs_reg_total,"124,692",6.1%
racial_geog_vs_reg_total,"148,931",7.3%
racial_geog_vs_50_percent,"147,899",7.2%
poverty_geog_vs_reg_total,"133,961",6.5%
poverty_geog_vs_50_percent,"221,180",10.8%
disability_geog_vs_reg_total,"97,116",4.7%


In [30]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
NOT in youth_geog_vs_reg_total,"220,287",10.7%
NOT in elderly_geog_vs_reg_total,"160,512",7.8%
NOT in english_geog_vs_reg_total,"142,347",6.9%
NOT in racial_geog_vs_reg_total,"125,081",6.1%
NOT in racial_geog_vs_50_percent,"132,674",6.5%
NOT in poverty_geog_vs_reg_total,"137,053",6.7%
NOT in poverty_geog_vs_50_percent,"133,685",6.5%
NOT in disability_geog_vs_reg_total,"169,294",8.3%


# Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.
Average accessible jobs are weighted averages based on parcel household population.

In [32]:
df = pd.read_csv(r'..\..\..\outputs\access\walk_bike_jobs_access.csv')

### Regional Average

In [33]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
"9,609","50,985",0.5%,2.5%


### Home County

In [34]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_county.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
King,"15,837","80,060",0.8%,3.9%
Kitsap,"1,145","6,990",0.1%,0.3%
Pierce,"2,190","16,116",0.1%,0.8%
Snohomish,"1,999","17,250",0.1%,0.8%


### Regional Growth Center

In [35]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_rgc.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
Auburn,"11,555","41,672",0.6%,2.0%
Ballard-Interbay,"13,278","80,607",0.6%,3.9%
Bellevue,"52,917","105,281",2.6%,5.1%
Bothell Canyon Park,"7,779","19,229",0.4%,0.9%
Bremerton,"12,005","29,786",0.6%,1.5%
Burien,"4,355","13,306",0.2%,0.6%
Cascade,"1,670","8,580",0.1%,0.4%
Duwamish,"28,832","248,710",1.4%,12.1%
Everett,"15,432","35,116",0.8%,1.7%


### Regional Geography

In [36]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_reg_geog.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
CitiesTowns,875,"7,225",0.0%,0.4%
Core,"2,956","27,722",0.1%,1.4%
HCT,"1,349","14,455",0.1%,0.7%
Metro,"25,391","120,825",1.2%,5.9%
Rural,105,"1,716",0.0%,0.1%
UU,350,"5,825",0.0%,0.3%


### Equity Geographies

In [37]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
youth_geog_vs_reg_total,"1,669","19,740",0.1%,1.0%
elderly_geog_vs_reg_total,"6,765","35,511",0.3%,1.7%
english_geog_vs_reg_total,"6,324","42,317",0.3%,2.1%
racial_geog_vs_reg_total,"13,776","61,381",0.7%,3.0%
racial_geog_vs_50_percent,"7,059","51,099",0.3%,2.5%
poverty_geog_vs_reg_total,"11,804","54,893",0.6%,2.7%
poverty_geog_vs_50_percent,"16,335","58,038",0.8%,2.8%
disability_geog_vs_reg_total,"11,227","41,920",0.5%,2.0%


In [38]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
NOT in youth_geog_vs_reg_total,"17,413","81,695",0.8%,4.0%
NOT in elderly_geog_vs_reg_total,"12,150","64,809",0.6%,3.2%
NOT in english_geog_vs_reg_total,"11,568","56,152",0.6%,2.7%
NOT in racial_geog_vs_reg_total,"6,235","42,566",0.3%,2.1%
NOT in racial_geog_vs_50_percent,"10,256","50,956",0.5%,2.5%
NOT in poverty_geog_vs_reg_total,"8,017","48,149",0.4%,2.3%
NOT in poverty_geog_vs_50_percent,"9,446","50,814",0.5%,2.5%
NOT in disability_geog_vs_reg_total,"8,204","58,854",0.4%,2.9%
